In [1]:
# %pip install plotly
# %pip install nbformat
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

os.getcwd()

'/Users/aneeshussain/Code/diss/analysis'

In [2]:
# Get the current directory
current_dir = os.getcwd()

# Change directory to the parent directory (diss)
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
os.chdir(parent_dir)

# Append the parent directory to the system path (optional, if you want to add it to the import path)
sys.path.append(parent_dir)

# Print the new current working directory to confirm the change
print(f"Changed directory to: {os.getcwd()}")

Changed directory to: /Users/aneeshussain/Code/diss


In [3]:
import pandas as pd

# Filepaths for each exchange
filepaths = {
    'binance': './data/processed/binance.csv',
    'okx': './data/processed/okx.csv',
    'bybit': './data/processed/bybit.csv'
}

# List of specified pairs
specified_pairs = ['BTCUSDCM', 'BTCUSDT', 'ETHUSDCM', 'ETHUSDT']

# Initialize a dictionary to store counts for each exchange
exchange_funding_stats = {}

# Loop through each exchange
for exchange, filepath in filepaths.items():
    # Load the data for the current exchange
    df = pd.read_csv(filepath)

    # Convert 'funding rate' to float
    df['funding rate'] = pd.to_numeric(df['funding rate'], errors='coerce')

    # Filter the DataFrame to only include the specified pairs
    filtered_df = df[df['pair'].isin(specified_pairs)]

    # Initialize dictionaries to store counts for this exchange
    positive_funding_counts = {}
    total_funding_counts = {}

    # Iterate through each specified pair
    for pair in specified_pairs:
        # Filter rows for the current pair
        pair_df = filtered_df[filtered_df['pair'] == pair]

        # Count positive funding rates
        positive_count = (pair_df['funding rate'] > 0).sum()
        total_count = len(pair_df)

        # Store the counts in the dictionaries
        positive_funding_counts[pair] = positive_count
        total_funding_counts[pair] = total_count

    # Create a DataFrame for this exchange to display the counts
    funding_stats = pd.DataFrame({
        'positive_count': positive_funding_counts,
        'total_count': total_funding_counts
    })

    # Calculate the percentage of positive funding rates
    funding_stats['positive_percentage'] = (funding_stats['positive_count'] / funding_stats['total_count']) * 100

    # Store the result for this exchange
    exchange_funding_stats[exchange] = funding_stats

# Display the results for each exchange
for exchange, stats in exchange_funding_stats.items():
    print(f"{exchange} Funding stats")
    print(stats)
    print("\n")

binance Funding stats
          positive_count  total_count  positive_percentage
BTCUSDCM            1435         1642            87.393423
BTCUSDT             1517         1642            92.387333
ETHUSDCM            1467         1642            89.342266
ETHUSDT             1539         1642            93.727162


okx Funding stats
          positive_count  total_count  positive_percentage
BTCUSDCM            1132         1642            68.940317
BTCUSDT             1203         1642            73.264312
ETHUSDCM            1163         1642            70.828258
ETHUSDT             1226         1642            74.665043


bybit Funding stats
          positive_count  total_count  positive_percentage
BTCUSDCM            1472         1642            89.646772
BTCUSDT             1494         1642            90.986602
ETHUSDCM            1481         1642            90.194884
ETHUSDT             1474         1642            89.768575




In [79]:
collateral_filepath = './results/test/collateral_log.csv'
collateral_df = pd.read_csv(collateral_filepath)

collateral_df['time'] = pd.to_datetime(collateral_df['time'])
collateral_df.set_index('time', inplace=True)

collateral_columns = [
    'binance_btc_collateral', 'binance_eth_collateral', 'binance_liquid_cash', 
    'okx_btc_collateral', 'okx_eth_collateral', 'okx_liquid_cash', 
    'bybit_btc_collateral', 'bybit_eth_collateral', 'bybit_liquid_cash'
]
funding_columns = ['binance_funding', 'okx_funding', 'bybit_funding']

initial_investment = 10000000
collateral_df['pnl_basis'] = collateral_df[collateral_columns].sum(axis=1) - initial_investment
collateral_df['pnl_funding'] = collateral_df[funding_columns].sum(axis=1)
collateral_df['total_pnl'] = collateral_df['pnl_basis'] + collateral_df['pnl_funding']

collateral_df['cumulative_pnl_basis_pct'] = (collateral_df['pnl_basis'] / initial_investment) * 100
collateral_df['cumulative_pnl_funding_pct'] = (collateral_df['pnl_funding'] / initial_investment) * 100
collateral_df['cumulative_total_pnl_pct'] = (collateral_df['total_pnl'] / initial_investment) * 100

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=collateral_df.index,
    y=collateral_df['pnl_basis'],
    mode='lines',
    name='P&L Basis',
    line=dict(width=2, color='blue'),
    hovertemplate='PnL Basis: $%{y:.2f}<br>Date: %{x}',
    yaxis='y1' 
))

fig.add_trace(go.Scatter(
    x=collateral_df.index,
    y=collateral_df['pnl_funding'],
    mode='lines',
    name='P&L Funding',
    line=dict(width=2, color='green'),
    hovertemplate='PnL Funding: $%{y:.2f}<br>Date: %{x}',
    yaxis='y1'
))

fig.add_trace(go.Scatter(
    x=collateral_df.index,
    y=collateral_df['total_pnl'],
    mode='lines',
    name='P&L',
    line=dict(width=2, color='orange'),
    hovertemplate='Total PnL: $%{y:.2f}<br>Date: %{x}',
    yaxis='y1'
))

fig.add_trace(go.Scatter(
    x=collateral_df.index,
    y=collateral_df['cumulative_pnl_basis_pct'],
    mode='lines',
    name='P&L Basis',
    line=dict(width=2, color='blue'),
    hovertemplate='PnL Basis: %{y:.2f}%<br>Date: %{x}',
    yaxis='y2',
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=collateral_df.index,
    y=collateral_df['cumulative_pnl_funding_pct'],
    mode='lines',
    name='P&L Funding',
    line=dict(width=2, color='green'),
    hovertemplate='PnL Funding: %{y:.2f}%<br>Date: %{x}',
    yaxis='y2',
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=collateral_df.index,
    y=collateral_df['cumulative_total_pnl_pct'],
    mode='lines',
    name='P&L',
    line=dict(width=2, color='orange'),
    hovertemplate='Total PnL: %{y:.2f}%<br>Date: %{x}',
    yaxis='y2',
    showlegend=False
))

fig.update_layout(
    title='Cumulative PnL Over Time',
    xaxis_title='Date',
    yaxis=dict(title='PnL ($)'), 
    yaxis2=dict(title='PnL (%)', overlaying='y', side='right'), 
    template='plotly_white',
    hovermode='x unified',    
    # margin=dict(l=50, r=100, t=50, b=50), 
    # autosize=True,  
    # width=1000, 
    # height=600, 
    legend=dict(
        x=1.05,
        y=1,
        xanchor='left',
        yanchor='top'
    )
)

fig.update_xaxes(
    tickformat='%b %Y',
    tickangle=0,
    showgrid=True,
    zeroline=True,
    showticklabels=True,
    dtick='M3'
)

fig.show()

In [41]:
ROLLING_WINDOW_DAYS = 30  # Change this value to set the window size

# Load the collateral data
collateral_filepath = './results/test/collateral_log.csv'
collateral_df = pd.read_csv(collateral_filepath)

# Convert 'time' to datetime and set as index
collateral_df['time'] = pd.to_datetime(collateral_df['time'])
collateral_df.set_index('time', inplace=True)

# Calculate instantaneous funding payments for each exchange (not cumulative)
collateral_df['binance_funding_value'] = collateral_df['binance_funding']
collateral_df['okx_funding_value'] = collateral_df['okx_funding']
collateral_df['bybit_funding_value'] = collateral_df['bybit_funding']

# Calculate individual components for each exchange
collateral_df['binance_total'] = (
    collateral_df['binance_btc_collateral'] +
    collateral_df['binance_eth_collateral'] +
    collateral_df['binance_liquid_cash'] +
    collateral_df['binance_funding_value']
)

collateral_df['okx_total'] = (
    collateral_df['okx_btc_collateral'] +
    collateral_df['okx_eth_collateral'] +
    collateral_df['okx_liquid_cash'] +
    collateral_df['okx_funding_value']
)

collateral_df['bybit_total'] = (
    collateral_df['bybit_btc_collateral'] +
    collateral_df['bybit_eth_collateral'] +
    collateral_df['bybit_liquid_cash'] +
    collateral_df['bybit_funding_value']
)

# Apply a rolling mean to smooth out the data based on ROLLING_WINDOW_DAYS
rolling_window = f'{ROLLING_WINDOW_DAYS}D'
for column in ['binance_total', 'okx_total', 'bybit_total']:
    collateral_df[column] = collateral_df[column].rolling(window=rolling_window).mean()

# Calculate percentage breakdown for each exchange
for exchange in ['binance', 'okx', 'bybit']:
    total = f'{exchange}_total'
    collateral_df[f'{exchange}_btc_pct'] = collateral_df[f'{exchange}_btc_collateral'].rolling(window=rolling_window).mean() / collateral_df[total]
    collateral_df[f'{exchange}_eth_pct'] = collateral_df[f'{exchange}_eth_collateral'].rolling(window=rolling_window).mean() / collateral_df[total]
    collateral_df[f'{exchange}_cash_pct'] = collateral_df[f'{exchange}_liquid_cash'].rolling(window=rolling_window).mean() / collateral_df[total]
    collateral_df[f'{exchange}_funding_pct'] = collateral_df[f'{exchange}_funding_value'].rolling(window=rolling_window).mean() / collateral_df[total]

# Create a function to generate stacked area plots for each exchange
def plot_exchange_breakdown(exchange_name):
    fig = go.Figure()
    # Liquid Cash at the bottom
    fig.add_trace(go.Scatter(
        x=collateral_df.index,
        y=collateral_df[f'{exchange_name}_cash_pct'],
        mode='lines',
        stackgroup='one',
        name='Liquid Cash'
    ))
    fig.add_trace(go.Scatter(
        x=collateral_df.index,
        y=collateral_df[f'{exchange_name}_btc_pct'],
        mode='lines',
        stackgroup='one',
        name='BTC Collateral'
    ))
    fig.add_trace(go.Scatter(
        x=collateral_df.index,
        y=collateral_df[f'{exchange_name}_eth_pct'],
        mode='lines',
        stackgroup='one',
        name='ETH Collateral'
    ))
    fig.add_trace(go.Scatter(
        x=collateral_df.index,
        y=collateral_df[f'{exchange_name}_funding_pct'],
        mode='lines',
        stackgroup='one',
        name='Funding Payments'
    ))
    
    # Update layout to customize appearance
    fig.update_layout(
        title=f'{exchange_name.capitalize()} Breakdown Over Time ({ROLLING_WINDOW_DAYS}-Day Rolling Average)',
        xaxis_title='Date',
        yaxis_title='Percentage of Exchange',
        template='plotly_white',
        xaxis=dict(tickformat='%b %Y'),
        yaxis=dict(tickformat='.0%', range=[0, 1]),
        hovermode='x unified'
    )

    return fig

# Generate and show plots for each exchange
fig_binance = plot_exchange_breakdown('binance')
fig_okx = plot_exchange_breakdown('okx')
fig_bybit = plot_exchange_breakdown('bybit')

# Display the figures
fig_binance.show()
fig_okx.show()
fig_bybit.show()

In [72]:
funding_filepath = './results/test/funding_log.csv' 
funding_df = pd.read_csv(funding_filepath)

# Convert 'time' to datetime format
funding_df['time'] = pd.to_datetime(funding_df['time'])

# Extract the month to aggregate funding payments
funding_df['month'] = funding_df['time'].dt.to_period('M')

# Calculate cumulative profit over time (using the original 'funding payment' column)
funding_df['cumulative_profit'] = funding_df['funding payment'].cumsum()

# Aggregate monthly funding payments for cumulative profit
monthly_cumulative_profit = funding_df.groupby('month')['cumulative_profit'].last().reset_index()

# Separate positive and negative funding payments before aggregation
funding_df['positive'] = funding_df['funding payment'].apply(lambda x: x if x >= 0 else 0)
funding_df['negative'] = funding_df['funding payment'].apply(lambda x: x if x < 0 else 0)

# Sum monthly positive and negative funding payments
monthly_funding = funding_df.groupby('month').agg({
    'positive': 'sum',
    'negative': 'sum'
}).reset_index()

# Convert 'month' back to a datetime for plotting (use the start of each month)
monthly_funding['month'] = monthly_funding['month'].dt.to_timestamp()


# Plotting using Plotly
fig = go.Figure()

# Add bars for positive funding payments
fig.add_trace(go.Bar(
    x=monthly_funding['month'],
    y=monthly_funding['positive'],
    name='Positive Funding Payments',
    marker_color='green',
))

# Add bars for negative funding payments
fig.add_trace(go.Bar(
    x=monthly_funding['month'],
    y=monthly_funding['negative'],
    name='Negative Funding Payments',
    marker_color='red',
))



# Update layout for the plot
fig.update_layout(
    title='Monthly Funding Payments with Cumulative Profit',
    xaxis_title='Month',
    yaxis_title='Funding Payment (USD)',
    template='plotly_white',
    barmode='relative',  # Allow bars to overlap properly
    hovermode='x unified',
)

# Format x-axis to display months clearly
fig.update_xaxes(
    tickformat='%b %Y',  # Format to show month (e.g., Jan 2023)
    tickangle=45,
    showgrid=True,
)

# Show the plot
fig.show()

In [54]:
trades_filepath = './results/test/trades_log.csv'
trades_df = pd.read_csv(trades_filepath)

# Convert 'open_time' and 'close_time' to datetime
trades_df['open_time'] = pd.to_datetime(trades_df['open_time'])
trades_df['close_time'] = pd.to_datetime(trades_df['close_time'])

# Filter for long positions
long_trades_df = trades_df[trades_df['position_type'] == 'long']

In [80]:
trades_filepath = './results/simple_threshold_open/trades_log.csv'
trades_df = pd.read_csv(trades_filepath)

# Convert 'open_time' and 'close_time' to datetime
trades_df['open_time'] = pd.to_datetime(trades_df['open_time'])
trades_df['close_time'] = pd.to_datetime(trades_df['close_time'])

# Filter for long positions
long_trades_df = trades_df[trades_df['position_type'] == 'long']

exchange_trade_count = long_trades_df['exchange'].value_counts()

# Display the count of trades on each exchange
print(exchange_trade_count)

exchange
okx        1128
bybit       353
binance     344
Name: count, dtype: int64


In [81]:
trades_filepath = './results/simple_reinvest_open/trades_log.csv'
trades_df = pd.read_csv(trades_filepath)

# Convert 'open_time' and 'close_time' to datetime
trades_df['open_time'] = pd.to_datetime(trades_df['open_time'])
trades_df['close_time'] = pd.to_datetime(trades_df['close_time'])

# Filter for long positions
long_trades_df = trades_df[trades_df['position_type'] == 'long']

exchange_trade_count = long_trades_df['exchange'].value_counts()

# Display the count of trades on each exchange
print(exchange_trade_count)

exchange
okx        1124
bybit       342
binance     337
Name: count, dtype: int64


In [86]:
trades_filepath = './results/complex_threshold_test/trades_log.csv'
trades_df = pd.read_csv(trades_filepath)

# Convert 'open_time' and 'close_time' to datetime
trades_df['open_time'] = pd.to_datetime(trades_df['open_time'])
trades_df['close_time'] = pd.to_datetime(trades_df['close_time'])

# Filter for long positions
long_trades_df = trades_df[trades_df['position_type'] == 'long']

exchange_trade_count = long_trades_df['exchange'].value_counts()

# Display the count of trades on each exchange
print(exchange_trade_count)

exchange
okx        1068
binance     335
bybit       335
Name: count, dtype: int64


In [87]:
trades_filepath = './results/complex_threshold_test1/trades_log.csv'
trades_df = pd.read_csv(trades_filepath)

# Convert 'open_time' and 'close_time' to datetime
trades_df['open_time'] = pd.to_datetime(trades_df['open_time'])
trades_df['close_time'] = pd.to_datetime(trades_df['close_time'])

# Filter for long positions
long_trades_df = trades_df[trades_df['position_type'] == 'long']

exchange_trade_count = long_trades_df['exchange'].value_counts()

# Display the count of trades on each exchange
print(exchange_trade_count)

exchange
okx        823
bybit      319
binance    306
Name: count, dtype: int64


In [88]:
trades_filepath = './results/complex_threshold_test2/trades_log.csv'
trades_df = pd.read_csv(trades_filepath)

# Convert 'open_time' and 'close_time' to datetime
trades_df['open_time'] = pd.to_datetime(trades_df['open_time'])
trades_df['close_time'] = pd.to_datetime(trades_df['close_time'])

# Filter for long positions
long_trades_df = trades_df[trades_df['position_type'] == 'long']

exchange_trade_count = long_trades_df['exchange'].value_counts()

# Display the count of trades on each exchange
print(exchange_trade_count)

exchange
okx        381
bybit      260
binance    236
Name: count, dtype: int64


In [90]:
trades_filepath = './results/complex_threshold_test3/trades_log.csv'
trades_df = pd.read_csv(trades_filepath)

# Convert 'open_time' and 'close_time' to datetime
trades_df['open_time'] = pd.to_datetime(trades_df['open_time'])
trades_df['close_time'] = pd.to_datetime(trades_df['close_time'])

# Filter for long positions
long_trades_df = trades_df[trades_df['position_type'] == 'long']

exchange_trade_count = long_trades_df['exchange'].value_counts()

# Display the count of trades on each exchange
print(exchange_trade_count)

exchange
okx        397
bybit      277
binance    253
Name: count, dtype: int64


In [95]:
trades_filepath = './results/complex_threshold_test8/trades_log.csv'
trades_df = pd.read_csv(trades_filepath)

# Convert 'open_time' and 'close_time' to datetime
trades_df['open_time'] = pd.to_datetime(trades_df['open_time'])
trades_df['close_time'] = pd.to_datetime(trades_df['close_time'])

# Filter for long positions
long_trades_df = trades_df[trades_df['position_type'] == 'long']

exchange_trade_count = long_trades_df['exchange'].value_counts()

# Display the count of trades on each exchange
print(exchange_trade_count)

exchange
okx        381
bybit      252
binance    229
Name: count, dtype: int64


In [92]:
trades_filepath = './results/complex_threshold_test5/trades_log.csv'
trades_df = pd.read_csv(trades_filepath)

# Convert 'open_time' and 'close_time' to datetime
trades_df['open_time'] = pd.to_datetime(trades_df['open_time'])
trades_df['close_time'] = pd.to_datetime(trades_df['close_time'])

# Filter for long positions
long_trades_df = trades_df[trades_df['position_type'] == 'long']

exchange_trade_count = long_trades_df['exchange'].value_counts()

# Display the count of trades on each exchange
print(exchange_trade_count)

exchange
okx        414
bybit      188
binance    162
Name: count, dtype: int64


In [93]:
trades_filepath = './results/complex_threshold_test6/trades_log.csv'
trades_df = pd.read_csv(trades_filepath)

# Convert 'open_time' and 'close_time' to datetime
trades_df['open_time'] = pd.to_datetime(trades_df['open_time'])
trades_df['close_time'] = pd.to_datetime(trades_df['close_time'])

# Filter for long positions
long_trades_df = trades_df[trades_df['position_type'] == 'long']

exchange_trade_count = long_trades_df['exchange'].value_counts()

# Display the count of trades on each exchange
print(exchange_trade_count)

exchange
okx        514
bybit      225
binance    207
Name: count, dtype: int64


In [8]:
collateral_filepath = './results/simple_hold_open_test/collateral_log.csv'
collateral_df = pd.read_csv(collateral_filepath)

collateral_df['time'] = pd.to_datetime(collateral_df['time'])
collateral_df.set_index('time', inplace=True)

collateral_columns = [
    'binance_btc_collateral', 'binance_eth_collateral', 'binance_liquid_cash', 
    'okx_btc_collateral', 'okx_eth_collateral', 'okx_liquid_cash', 
    'bybit_btc_collateral', 'bybit_eth_collateral', 'bybit_liquid_cash'
]
funding_columns = ['binance_funding', 'okx_funding', 'bybit_funding']

unrealised_pnl_columns = ['binance_unrealised_pnl', 'okx_unrealised_pnl', 'bybit_unrealised_pnl']

initial_investment = 10000000
collateral_df['pnl_basis'] = collateral_df[collateral_columns].sum(axis=1) - initial_investment
collateral_df['pnl_funding'] = collateral_df[funding_columns].sum(axis=1)
collateral_df['total_pnl'] = collateral_df['pnl_basis'] + collateral_df['pnl_funding']

collateral_df['cumulative_pnl_basis_pct'] = (collateral_df['pnl_basis'] / initial_investment) * 100
collateral_df['cumulative_pnl_funding_pct'] = (collateral_df['pnl_funding'] / initial_investment) * 100
collateral_df['cumulative_total_pnl_pct'] = (collateral_df['total_pnl'] / initial_investment) * 100
collateral_df['cumulative_unrealised_pnl'] = collateral_df[unrealised_pnl_columns].sum(axis=1)


fig = go.Figure()

fig.add_trace(go.Scatter(
    x=collateral_df.index,
    y=collateral_df['pnl_basis'],
    mode='lines',
    name='P&L Basis',
    line=dict(width=2, color='blue'),
    hovertemplate='PnL Basis: $%{y:.2f}<br>Date: %{x}',
    yaxis='y1' 
))

fig.add_trace(go.Scatter(
    x=collateral_df.index,
    y=collateral_df['pnl_funding'],
    mode='lines',
    name='P&L Funding',
    line=dict(width=2, color='green'),
    hovertemplate='PnL Funding: $%{y:.2f}<br>Date: %{x}',
    yaxis='y1'
))

fig.add_trace(go.Scatter(
    x=collateral_df.index,
    y=collateral_df['total_pnl'],
    mode='lines',
    name='P&L',
    line=dict(width=2, color='orange'),
    hovertemplate='Total PnL: $%{y:.2f}<br>Date: %{x}',
    yaxis='y1'
))

# fig.add_trace(go.Scatter(
#     x=collateral_df.index,
#     y=collateral_df['cumulative_pnl_basis_pct'],
#     mode='lines',
#     name='P&L Basis',
#     line=dict(width=2, color='blue'),
#     hovertemplate='PnL Basis: %{y:.2f}%<br>Date: %{x}',
#     yaxis='y2',
#     showlegend=False
# ))

# fig.add_trace(go.Scatter(
#     x=collateral_df.index,
#     y=collateral_df['cumulative_pnl_funding_pct'],
#     mode='lines',
#     name='P&L Funding',
#     line=dict(width=2, color='green'),
#     hovertemplate='PnL Funding: %{y:.2f}%<br>Date: %{x}',
#     yaxis='y2',
#     showlegend=False
# ))

# fig.add_trace(go.Scatter(
#     x=collateral_df.index,
#     y=collateral_df['cumulative_total_pnl_pct'],
#     mode='lines',
#     name='P&L',
#     line=dict(width=2, color='orange'),
#     hovertemplate='Total PnL: %{y:.2f}%<br>Date: %{x}',
#     yaxis='y2',
#     showlegend=False
# ))

fig.add_trace(go.Scatter(
    x=collateral_df.index,
    y=collateral_df['cumulative_unrealised_pnl'],
    mode='lines',
    name='Unrealised PnL',
    line=dict(width=2, color='purple'),
    hovertemplate='Unrealised PnL: $%{y:.2f}<br>Date: %{x}',
    yaxis='y1'
))

fig.update_layout(
    title='Cumulative PnL Over Time',
    xaxis_title='Date',
    yaxis=dict(title='PnL ($)'), 
    yaxis2=dict(title='PnL (%)', overlaying='y', side='right'), 
    template='plotly_white',
    hovermode='x unified',    
    # margin=dict(l=50, r=100, t=50, b=50), 
    # autosize=True,  
    # width=1000, 
    # height=600, 
    legend=dict(
        x=1.05,
        y=1,
        xanchor='left',
        yanchor='top'
    )
)

fig.update_xaxes(
    tickformat='%b %Y',
    tickangle=0,
    showgrid=True,
    zeroline=True,
    showticklabels=True,
    dtick='M3'
)

fig.show()